In [1]:
import torch
from torch.optim import Adam
from tqdm import tqdm

import scipy
from scipy.io import loadmat
from scipy.io import savemat
import numpy as np

import matplotlib.pyplot as plt


import seaborn as sns
color_names = ["windows blue", "red", "amber", "faded green"]
colors = sns.xkcd_palette(color_names)
sns.set_style("white")
sns.set_context("talk")


def frobenius_norm(tensor):
    return torch.norm(tensor, 'fro')


def loss_function(X, Y, C):
    return frobenius_norm(X - C @ Y @ C.inverse())


def cayley_transform(A):
    I = torch.eye(A.size(0), dtype=A.dtype, device=A.device)
    return torch.linalg.solve(I + A, I - A)

def cayley_numpy(A):
    I =np.eye((Aa-Aa.T).shape[0])
    return np.linalg.solve(I+A,I-A)

learning_rate = 0.01
max_iterations = 1500



# Now do DSA with A~ high components models with ncomps=116 (80% variance in PODs)

Run DSA

In [2]:
DataFold='/Users/user/PycharmProjects/PacManMain/data/DMDmat/UPDATED/AllComponents/'
Xall=loadmat(DataFold+'/X_Amatrix_fullAllComps_fullwin.mat')['XA']
Yall=loadmat(DataFold+'/Y_Amatrix_fullAllComps_fullwin.mat')['YA']


X=torch.from_numpy(np.float32(Xall))
Y=torch.from_numpy(np.float32(Yall))


max_iterations=3000

n = X.size(0)

# Initialize a skew-symmetric matrix A

A = torch.randn((n, n), requires_grad=True)
A.data = A.data - A.data.T  # Making A skew-symmetric but preserving leaf status
# Use ADAM optimizer
optimizer = Adam([A], lr=learning_rate)

for iteration in range(max_iterations):
    optimizer.zero_grad()

    # Ensure C is orthogonal via Cayley transform of skew-symmetric A
    C = cayley_transform(A)
    loss = loss_function(X, Y, C)

    # Compute gradients
    loss.backward()

    # Update A using gradients
    optimizer.step()

    if iteration % 100 == 0:
        print(f'Iteration {iteration}, Loss: {loss.item()}')

# Final optimized C
C_optimized = cayley_transform(A)
C_optimized=C_optimized.detach().numpy()
Optimized_loss=loss.detach().numpy()

#compute angular
num = np.trace(X.detach().numpy().T @ C_optimized @ Y.detach().numpy() @ np.linalg.inv(C_optimized))


denom = torch.norm(X,p = 'fro')*torch.norm(Y,p = 'fro')
denom=denom.detach().numpy()
angoptimalscore = np.cos(np.arccos(num/denom))


/Users/user/anaconda3/envs/main/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Iteration 0, Loss: 1.9423234462738037
Iteration 100, Loss: 1.3587335348129272
Iteration 200, Loss: 1.148522973060608
Iteration 300, Loss: 1.0370972156524658
Iteration 400, Loss: 0.9621788263320923
Iteration 500, Loss: 0.9103868007659912
Iteration 600, Loss: 0.8693132400512695
Iteration 700, Loss: 0.8358630537986755
Iteration 800, Loss: 0.8070039749145508
Iteration 900, Loss: 0.7812249064445496
Iteration 1000, Loss: 0.7585174441337585
Iteration 1100, Loss: 0.739311158657074
Iteration 1200, Loss: 0.7236678004264832
Iteration 1300, Loss: 0.7058394551277161
Iteration 1400, Loss: 0.6933740973472595
Iteration 1500, Loss: 0.6800132989883423
Iteration 1600, Loss: 0.6685836315155029
Iteration 1700, Loss: 0.6560708284378052
Iteration 1800, Loss: 0.6435085535049438
Iteration 1900, Loss: 0.6355312466621399
Iteration 2000, Loss: 0.6235182285308838
Iteration 2100, Loss: 0.6170229315757751
Iteration 2200, Loss: 0.6062996983528137
Iteration 2300, Loss: 0.5995985269546509
Iteration 2400, Loss: 0.592819

## Perm with random x 50

In [3]:

DataFold='/Users/user/PycharmProjects/PacManMain/data/DMDmat/UPDATED/AllComponents/'
Xall=loadmat(DataFold+'/X_Amatrix_fullAllComps_fullwin.mat')['XA']

C_on_perm=np.empty([Xall.shape[0],Xall.shape[0],50])
loss_on_perm=np.zeros((50))
angpermscore=np.zeros((50))
for perm in range(50):
    Xall=loadmat(DataFold+'/X_Amatrix_fullAllComps_fullwin.mat')['XA']
    Yall=loadmat(DataFold+'/Y_Amatrix_fullAllComps_fullwin.mat')['YA']

    X=torch.from_numpy(np.float32(Xall))
    n = X.size(0)

    #apply rotation using random skew symmetric orthogonal transform
    Aa=np.random.randn(n,n)
    Yall=cayley_numpy(Aa-Aa.T)*Yall * scipy.linalg.inv(cayley_numpy(Aa-Aa.T))

    Y=torch.from_numpy(np.float32(Yall))

    # Initialize a skew-symmetric matrix A

    A = torch.randn((n, n), requires_grad=True)
    A.data = A.data - A.data.T  # Making A skew-symmetric but preserving leaf status
    # Use ADAM optimizer
    optimizer = Adam([A], lr=learning_rate)

    for iteration in range(max_iterations):
        optimizer.zero_grad()

        # Ensure C is orthogonal via Cayley transform of skew-symmetric A
        C = cayley_transform(A)
        loss = loss_function(X, Y, C)

        # Compute gradients
        loss.backward()

        # Update A using gradients
        optimizer.step()

        if iteration % 100 == 0:
            print(f'Iteration {iteration}, Loss: {loss.item()}')

    # Final optimized C
    C_perm = cayley_transform(A)
    C_perm=C_perm.detach().numpy()
    C_on_perm[:,:,perm]=C_perm
    losstmp = loss.detach().numpy()
    loss_on_perm[perm]=losstmp

    #compute angular
    num = np.trace(X.detach().numpy().T @ C_perm @ Y.detach().numpy() @ np.linalg.inv(C_perm))


    denom = torch.norm(X,p = 'fro')*torch.norm(Y,p = 'fro')
    denom=denom.detach().numpy()
    angpermscore[perm] = np.cos(np.arccos(num/denom))


Iteration 0, Loss: 2.814499616622925
Iteration 100, Loss: 2.695152759552002
Iteration 200, Loss: 2.6193981170654297
Iteration 300, Loss: 2.5923919677734375
Iteration 400, Loss: 2.57241153717041
Iteration 500, Loss: 2.5563366413116455
Iteration 600, Loss: 2.5441765785217285
Iteration 700, Loss: 2.533590793609619
Iteration 800, Loss: 2.5256547927856445
Iteration 900, Loss: 2.518123149871826
Iteration 1000, Loss: 2.511741876602173
Iteration 1100, Loss: 2.506168842315674
Iteration 1200, Loss: 2.500749349594116
Iteration 1300, Loss: 2.4972872734069824
Iteration 1400, Loss: 2.4928553104400635
Iteration 1500, Loss: 2.4894542694091797
Iteration 1600, Loss: 2.487091064453125
Iteration 1700, Loss: 2.483168363571167
Iteration 1800, Loss: 2.4805798530578613
Iteration 1900, Loss: 2.478231430053711
Iteration 2000, Loss: 2.4758238792419434
Iteration 2100, Loss: 2.474381685256958
Iteration 2200, Loss: 2.471341133117676
Iteration 2300, Loss: 2.470160722732544
Iteration 2400, Loss: 2.46897029876709
Iter

/var/folders/tv/_yg34tj15s306l4m7k_wf0qw0000gq/T/ipykernel_66893/267177544.py:56: RuntimeWarning: invalid value encountered in arccos
  angpermscore[perm] = np.cos(np.arccos(num/denom))
/var/folders/tv/_yg34tj15s306l4m7k_wf0qw0000gq/T/ipykernel_66893/267177544.py:56: RuntimeWarning: invalid value encountered in cos
  angpermscore[perm] = np.cos(np.arccos(num/denom))


Iteration 0, Loss: 3.003929376602173
Iteration 100, Loss: 2.868960380554199
Iteration 200, Loss: 2.879561185836792
Iteration 300, Loss: 2.8424997329711914
Iteration 400, Loss: 2.8255341053009033
Iteration 500, Loss: 2.812756061553955
Iteration 600, Loss: 2.8197414875030518
Iteration 700, Loss: 2.7923316955566406
Iteration 800, Loss: 2.7933003902435303
Iteration 900, Loss: 2.780036687850952
Iteration 1000, Loss: 2.771315574645996
Iteration 1100, Loss: 2.7740960121154785
Iteration 1200, Loss: 2.758230686187744
Iteration 1300, Loss: 2.7564501762390137
Iteration 1400, Loss: 2.7601683139801025
Iteration 1500, Loss: 2.7563204765319824
Iteration 1600, Loss: 2.7418081760406494
Iteration 1700, Loss: 2.7385354042053223
Iteration 1800, Loss: 2.7335829734802246
Iteration 1900, Loss: 2.7298953533172607
Iteration 2000, Loss: 2.7265751361846924
Iteration 2100, Loss: 2.7245676517486572
Iteration 2200, Loss: 2.7302308082580566
Iteration 2300, Loss: 2.7196075916290283
Iteration 2400, Loss: 2.71635174751

In [4]:
Xall=loadmat(DataFold+'/X_Amatrix_fullAllComps_fullwin.mat')['XA']
Yall=loadmat(DataFold+'/Y_Amatrix_fullAllComps_fullwin.mat')['YA']




mdic = {'optimizedC':[C_optimized],'optoloss':[Optimized_loss],'angularoptimal': [angoptimalscore],'loss_on_perm_random': [loss_on_perm],'C_on_perm' : [C_on_perm],'angperscore': [angpermscore],'XA':[Xall],'YA':[Yall]}
savemat(DataFold+'/C_optimized_full_dmd_full_win_ACC.mat',mdic)



## Perm with eigen maintained

# Now do DSA in PMD with A~ high components models with ncomps=197 (80% variance in PODs)

In [5]:
DataFold='/Users/user/PycharmProjects/PacManMain/data/DMDmat/UPDATED/AllComponents/'
Xall=loadmat(DataFold+'/X_Amatrix_fullAllComps_fullwin_pmd.mat')['XA']
Yall=loadmat(DataFold+'/Y_Amatrix_fullAllComps_fullwin_pmd.mat')['YA']


X=torch.from_numpy(np.float32(Xall))
Y=torch.from_numpy(np.float32(Yall))


max_iterations=3000

n = X.size(0)

# Initialize a skew-symmetric matrix A

A = torch.randn((n, n), requires_grad=True)
A.data = A.data - A.data.T  # Making A skew-symmetric but preserving leaf status
# Use ADAM optimizer
optimizer = Adam([A], lr=learning_rate)

for iteration in range(max_iterations):
    optimizer.zero_grad()

    # Ensure C is orthogonal via Cayley transform of skew-symmetric A
    C = cayley_transform(A)
    loss = loss_function(X, Y, C)

    # Compute gradients
    loss.backward()

    # Update A using gradients
    optimizer.step()

    if iteration % 100 == 0:
        print(f'Iteration {iteration}, Loss: {loss.item()}')

# Final optimized C
C_optimized = cayley_transform(A)
C_optimized=C_optimized.detach().numpy()
Optimized_loss=loss.detach().numpy()

#compute angular
num = np.trace(X.detach().numpy().T @ C_optimized @ Y.detach().numpy() @ np.linalg.inv(C_optimized))


denom = torch.norm(X,p = 'fro')*torch.norm(Y,p = 'fro')
denom=denom.detach().numpy()
angoptimalscore = np.cos(np.arccos(num/denom))

Iteration 0, Loss: 2.9436275959014893
Iteration 100, Loss: 2.3967959880828857
Iteration 200, Loss: 2.28698468208313
Iteration 300, Loss: 2.1588640213012695
Iteration 400, Loss: 2.0603926181793213
Iteration 500, Loss: 1.9848281145095825
Iteration 600, Loss: 1.9109870195388794
Iteration 700, Loss: 1.8531900644302368
Iteration 800, Loss: 1.799070954322815
Iteration 900, Loss: 1.75983464717865
Iteration 1000, Loss: 1.7113656997680664
Iteration 1100, Loss: 1.6736339330673218
Iteration 1200, Loss: 1.6380637884140015
Iteration 1300, Loss: 1.617935061454773
Iteration 1400, Loss: 1.5723769664764404
Iteration 1500, Loss: 1.5464047193527222
Iteration 1600, Loss: 1.528183937072754
Iteration 1700, Loss: 1.4933598041534424
Iteration 1800, Loss: 1.471987247467041
Iteration 1900, Loss: 1.454038381576538
Iteration 2000, Loss: 1.4329756498336792
Iteration 2100, Loss: 1.410189151763916
Iteration 2200, Loss: 1.3932673931121826
Iteration 2300, Loss: 1.3697278499603271
Iteration 2400, Loss: 1.35756444931030

Do perm DSA with PMD

In [6]:
DataFold='/Users/user/PycharmProjects/PacManMain/data/DMDmat/UPDATED/AllComponents/'
Xall=loadmat(DataFold+'/X_Amatrix_fullAllComps_fullwin_pmd.mat')['XA']

C_on_perm=np.empty([Xall.shape[0],Xall.shape[0],50])
loss_on_perm=np.zeros((50))
angpermscore=np.zeros((50))
for perm in range(50):
    Xall=loadmat(DataFold+'/X_Amatrix_fullAllComps_fullwin_pmd.mat')['XA']
    Yall=loadmat(DataFold+'/Y_Amatrix_fullAllComps_fullwin_pmd.mat')['YA']

    X=torch.from_numpy(np.float32(Xall))
    n = X.size(0)

    #apply rotation using random skew symmetric orthogonal transform
    Aa=np.random.randn(n,n)
    Yall=cayley_numpy(Aa-Aa.T)*Yall * scipy.linalg.inv(cayley_numpy(Aa-Aa.T))

    Y=torch.from_numpy(np.float32(Yall))

    # Initialize a skew-symmetric matrix A

    A = torch.randn((n, n), requires_grad=True)
    A.data = A.data - A.data.T  # Making A skew-symmetric but preserving leaf status
    # Use ADAM optimizer
    optimizer = Adam([A], lr=learning_rate)

    for iteration in range(max_iterations):
        optimizer.zero_grad()

        # Ensure C is orthogonal via Cayley transform of skew-symmetric A
        C = cayley_transform(A)
        loss = loss_function(X, Y, C)

        # Compute gradients
        loss.backward()

        # Update A using gradients
        optimizer.step()

        if iteration % 100 == 0:
            print(f'Iteration {iteration}, Loss: {loss.item()}')

    # Final optimized C
    C_perm = cayley_transform(A)
    C_perm=C_perm.detach().numpy()
    C_on_perm[:,:,perm]=C_perm
    losstmp = loss.detach().numpy()
    loss_on_perm[perm]=losstmp

    #compute angular
    num = np.trace(X.detach().numpy().T @ C_perm @ Y.detach().numpy() @ np.linalg.inv(C_perm))


    denom = torch.norm(X,p = 'fro')*torch.norm(Y,p = 'fro')
    denom=denom.detach().numpy()
    angpermscore[perm] = np.cos(np.arccos(num/denom))

Iteration 0, Loss: 3.1339266300201416
Iteration 100, Loss: 3.0806498527526855
Iteration 200, Loss: 3.1003472805023193
Iteration 300, Loss: 3.0249555110931396
Iteration 400, Loss: 3.080049753189087
Iteration 500, Loss: 3.0962111949920654
Iteration 600, Loss: 3.0913684368133545
Iteration 700, Loss: 2.977271556854248
Iteration 800, Loss: 3.0038652420043945
Iteration 900, Loss: 2.924264669418335
Iteration 1000, Loss: 3.0446360111236572
Iteration 1100, Loss: 2.935485363006592
Iteration 1200, Loss: 2.899998188018799
Iteration 1300, Loss: 3.087583065032959
Iteration 1400, Loss: 3.0119309425354004
Iteration 1500, Loss: 3.077054500579834
Iteration 1600, Loss: 3.0077450275421143
Iteration 1700, Loss: 3.069413900375366
Iteration 1800, Loss: 3.00087571144104
Iteration 1900, Loss: 3.0880911350250244
Iteration 2000, Loss: 3.071841239929199
Iteration 2100, Loss: 3.040009021759033
Iteration 2200, Loss: 2.987030029296875
Iteration 2300, Loss: 3.050919771194458
Iteration 2400, Loss: 2.948763847351074
It

/var/folders/tv/_yg34tj15s306l4m7k_wf0qw0000gq/T/ipykernel_66893/3875915117.py:56: RuntimeWarning: invalid value encountered in arccos
  angpermscore[perm] = np.cos(np.arccos(num/denom))
/var/folders/tv/_yg34tj15s306l4m7k_wf0qw0000gq/T/ipykernel_66893/3875915117.py:56: RuntimeWarning: invalid value encountered in cos
  angpermscore[perm] = np.cos(np.arccos(num/denom))


Iteration 0, Loss: 3.2317795753479004
Iteration 100, Loss: 3.209181070327759
Iteration 200, Loss: 3.1807334423065186
Iteration 300, Loss: 3.186981439590454
Iteration 400, Loss: 3.1867892742156982
Iteration 500, Loss: 3.1216559410095215
Iteration 600, Loss: 3.2026233673095703
Iteration 700, Loss: 3.19620943069458
Iteration 800, Loss: 3.1901071071624756
Iteration 900, Loss: 3.181532382965088
Iteration 1000, Loss: 3.1624958515167236
Iteration 1100, Loss: 3.1092963218688965
Iteration 1200, Loss: 3.118081569671631
Iteration 1300, Loss: 3.189025640487671
Iteration 1400, Loss: 3.201256036758423
Iteration 1500, Loss: 3.189573287963867
Iteration 1600, Loss: 3.1615066528320312
Iteration 1700, Loss: 3.185777187347412
Iteration 1800, Loss: 3.170750617980957
Iteration 1900, Loss: 3.1431548595428467
Iteration 2000, Loss: 3.1628127098083496
Iteration 2100, Loss: 3.1770386695861816
Iteration 2200, Loss: 3.1658101081848145
Iteration 2300, Loss: 3.1485402584075928
Iteration 2400, Loss: 3.105380535125732

In [7]:
Xall=loadmat(DataFold+'/X_Amatrix_fullAllComps_fullwin_pmd.mat')['XA']
Yall=loadmat(DataFold+'/Y_Amatrix_fullAllComps_fullwin_pmd.mat')['YA']



mdic = {'optimizedC':[C_optimized],'optoloss':[Optimized_loss],'angularoptimal': [angoptimalscore],'loss_on_perm_random': [loss_on_perm],'C_on_perm' : [C_on_perm],'angperscore': [angpermscore],'Xall':[Xall],'Yall':[Yall]}
savemat(DataFold+'/C_optimized_full_dmd_PMD.mat',mdic)


xlim=(-.6, .6)
ylim=(-.6, .6)
nxpts=9
nypts=9
alpha=0.8
ax=None
figsize=(3, 3)


x = np.linspace(*xlim, nxpts)
y = np.linspace(*ylim, nypts)
X, Y = np.meshgrid(x, y)
xy = np.column_stack((X.ravel(), Y.ravel()))


fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(111)

dxydt = xy.dot(Xall[0:2,0:2])-xy

ax.quiver(xy[:, 0], xy[:, 1], dxydt[:, 0], dxydt[:, 1],color=colors[0])


dxydt = xy.dot(Yall[0:2,0:2])-xy

ax.quiver(xy[:, 0], xy[:, 1], dxydt[:, 0], dxydt[:, 1],  color=colors[1])


# split plot

Yprime=C_optimized@Yall@np.linalg.inv(C_optimized)

fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(111)

dxydt = xy.dot(Xall[0:2,0:2])-xy

ax.quiver(xy[:, 0], xy[:, 1], dxydt[:, 0], dxydt[:, 1],color=colors[0])


dxydt = xy.dot(Yprime[0:2,0:2])-xy

ax.quiver(xy[:, 0], xy[:, 1], dxydt[:, 0], dxydt[:, 1],  color=colors[1])
plt.show()